In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/simon-easy-ocr-data'
#role = 'arn:aws:iam::294038372338:role/hunkimSagemaker'

In [2]:
#Make a dummy model.tar.gz
import tarfile
import io

with tarfile.open('model.tar.gz', "w:gz") as tar:
  dummy = b"dummy"
  info = tarfile.TarInfo('dummy.bin')
  info.size = len(dummy)
  tar.addfile(info, io.BytesIO(dummy))

In [3]:
dummy_model = sagemaker_session.upload_data(path='model.tar.gz', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(dummy_model))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-611238842315/sagemaker/simon-easy-ocr-data/model.tar.gz


In [6]:
from sagemaker.pytorch import PyTorchModel

# FIXME: Need to upload Dummy model (perhaps only model.targ.gz)
print(dummy_model)
model = PyTorchModel(model_data=dummy_model,
                     role=role,
                     framework_version='1.5.0',
                     py_version = "py3",
                     entry_point='ocr_infer.py',
                     source_dir='code')

s3://sagemaker-us-east-1-611238842315/sagemaker/simon-easy-ocr-data/model.tar.gz


In [7]:
%%time
predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

------------------!CPU times: user 559 ms, sys: 22.3 ms, total: 582 ms
Wall time: 9min 3s


In [8]:
# Get the end point
endpoint = predictor.endpoint  
print(endpoint)
print("See the logs at", "https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/"+ endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


pytorch-inference-2021-06-02-17-09-49-490
See the logs at https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-2021-06-02-17-09-49-490


In [9]:
%%time
import boto3
import json

client = boto3.client('sagemaker-runtime')

img = open('code/example.jpg', 'rb').read()

response = client.invoke_endpoint(
    EndpointName=endpoint, 
    ContentType='application/x-image', 
    #Accept="application/json" ,
    Body=bytearray(img)
)

print(response['Body'].read()) 

b'[[[[135, 21], [223, 21], [223, 61], [135, 61]], "\\uae40\\uc9c4\\uc911", 0.9923191129311097], [[[138, 68], [206, 68], [206, 94], [138, 94]], "6 hrs", 0.8211658289798127], [[[799, 151], [937, 151], [937, 189], [799, 189]], "OBSHD", 0.7918025384202817], [[[779, 435], [815, 435], [815, 473], [779, 473]], "N", 0.8565304563530987], [[[256, 614], [758, 614], [758, 670], [256, 670]], "\\uc798 \\ub9de\\uc9c0 \\uc548\\ub294 \\uc0ac\\ub78c\\ub4e4 \\ubc11\\uc5d0\\uc11c", 0.2126438874973759], [[[360, 670], [656, 670], [656, 726], [360, 726]], "\\uc77c\\ud558\\uc9c0\\ub17c \\ub9c8\\uc138\\uc694", 0.6879173934738305], [[[274, 800], [344, 800], [344, 824], [274, 824]], "OBSHD", 0.8634788661817754], [[[624, 800], [678, 800], [678, 824], [624, 824]], "OBS |", 0.19923878283537824], [[[964, 800], [1012, 800], [1012, 824], [964, 824]], "OBS", 0.5149565427408263], [[[44, 1032], [220, 1032], [220, 1062], [44, 1062]], "\\ubbf8\\uc74c\\uc774 \\ub04c\\ub9ac\\ub294 \\uc77c\\uc744", 0.2827901428487838], [[[397

In [10]:
sagemaker_session.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [21]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '15GPHV1YGHW267EP',
   'HostId': 'yz5meepruJt5a5npJ4ot1zlrRuILk/s1xS0fBZA6WQagcNFCsDonm4HuHyWMeL9vMEgsmc76HRw=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'yz5meepruJt5a5npJ4ot1zlrRuILk/s1xS0fBZA6WQagcNFCsDonm4HuHyWMeL9vMEgsmc76HRw=',
    'x-amz-request-id': '15GPHV1YGHW267EP',
    'date': 'Wed, 02 Jun 2021 17:36:01 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'pytorch-inference-2021-06-02-17-09-48-817/model.tar.gz'},
   {'Key': 'sagemaker/simon-easy-ocr-data/model.tar.gz'}]}]